# Redes Neuronales

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from scipy import optimize

In [3]:
csv_x = 'xy/x.csv'
csv_y = 'xy/y.csv'
csv_theta1 = 'theta/theta1.csv'
csv_theta2 = 'theta/theta2.csv'

### sigmoide

In [4]:
def sigmoid(a):
    return 1.0 / (1 + np.exp(-a))

### Convert to Array

In [5]:
X = np.genfromtxt(csv_x, delimiter=',')
X.shape

(5000, 400)

In [6]:
Theta1 = np.genfromtxt(csv_theta1, delimiter=',')
Theta1.shape

(25, 401)

In [7]:
Theta2 = np.genfromtxt(csv_theta2, delimiter=',')
Theta2.shape

(10, 26)

In [8]:
Y = np.genfromtxt(csv_y, delimiter=',')
Y.shape

(5000,)

### Fordward

In [9]:
def forward(x, theta1, theta2):
    #capa 1
    bias1 = np.ones((x.shape[0],1))
    a1 = np.append(bias1,x, axis=1)
    
    #capa2
    z2 = np.dot(a1,theta1.T)
    a2 = sigmoid(z2)
    a2 = np.append(np.ones((a2.shape[0],1)),a2, axis=1)
    
    #capa3
    z3 = np.dot(a2,theta2.T)
    a3 = sigmoid(z3)
    #result = np.argmax(a3,axis=1).T
    result={'a1': a1, 'a2':a2, 'a3':a3, 'z2':z2, 'z3':z3}
    
    return result

In [10]:
def precision(res, y):
    contador = 0
    for i in range(res.shape[0]) :
        if ((res[i]+1) == Y[i]):
            contador +=1
    
    return (contador*100.0)/res.shape[0]

In [11]:
def init_weight(input_layer, out_layer):
    return np.random.rand(out_layer,input_layer+1) # + el bias

In [12]:
#vectoriza lo que hay en Y
def create_Y(y, out_size, m):
    yn = np.zeros((m, out_size))
    for i in range(m):
        yn[i][int(y[i])%10] = 1
    return yn

### Gradient Computation

In [13]:
def gradiente_prima(z):
    gz = sigmoid(z)
    return gz * (1-gz);

In [14]:
def compute_gradient_reg(ford, yv, m, theta1, theta2):    
    a1 = ford['a1']
    a2 = ford['a2']
    a3 = ford['a3']
    z2 = ford['z2']
    z3 = ford['z3']
        
    d3 = a3 - yv
    d2 = np.dot(d3,theta2) * gradiente_prima(np.append(np.ones((z2.shape[0],1)),z2, axis=1)) #5000*26
    d2 = d2[:,1:] #quita la 1era columna
    delta1 = np.dot(d2.T,a1)
    delta2 = np.dot(d3.T,a2)
    
    grad1 = delta1/m
    grad2 = delta2/m
    #grad = np.c_[grad1.reshape(1,grad1.size),grad2.reshape(1,grad2.size)].T
    
    return grad1,grad2

In [15]:
def compute_cost_reg(theta1, theta2, X, Y, parametroLambda):
    yNew = create_Y(Y, 10, X.shape[0])
    m = X.shape[0]
    a1 = np.c_[np.ones((m,1)),X]
    
    z2 = np.dot(a1,theta1.T)
    a2 = np.c_[np.ones((z2.shape[0],1)),sigmoid(z2)]
    
    z3 = np.dot(a2,theta2.T)
    a3 = sigmoid(z3)
    
    h = a3
    reg = np.sum(np.power(theta1[:,1:],2))+np.sum(np.power(theta2[:,1:],2))
    J = (-np.sum(yNew * np.log(h)) - np.sum((1-yNew)* np.log(1-h)) + parametroLambda*reg/2)/m
    
    return J

In [16]:
cost = compute_cost_reg(Theta1, Theta2, X, Y, 0.01)
cost

10.442421079717274

In [17]:
def compute_cost(A3, Y):
    m = Y.shape[0] #(5000,10)
    cost = (- 1 / m) * np.sum(np.sum(Y * np.log(A3) + (1 - Y) * (np.log(1 - A3)), axis=1), axis=0)
    return cost

### NN

In [18]:
def session(x, y, hidden_layer, t1_, t2_, iteraciones, rate=1.5):
    input_layer = x.shape[1]
    out_layer = y.shape[1]
    m = x.shape[0]
    
    w1 = init_weight(input_layer, hidden_layer)
    w2 = init_weight(hidden_layer, out_layer)
    #w1=t1_
    #w2=t2_
    for i in range(iteraciones):
        ford = forward(x,w1,w2)
        cost = compute_cost(ford['a3'], y)
        grad1,grad2 = compute_gradient_reg(ford, y, m, w1, w2)
        
        w1 = w1 - (rate * grad1)
        w2 = w2 - (rate * grad2)
        
        if i % 100 == 0:
            print (i," iteracion con costo: ",cost)
    return w1,w2
    

In [19]:
Y_vec = create_Y(Y, 10, X.shape[0])
W1,W2 = session(X,Y_vec,25,Theta1,Theta2,2000)

0  iteracion con costo:  117.97497957757987
100  iteracion con costo:  4.376518846756558
200  iteracion con costo:  4.192341650232638
300  iteracion con costo:  4.9427708645907
400  iteracion con costo:  5.220238602488884
500  iteracion con costo:  4.854993963734867
600  iteracion con costo:  4.589799921109798
700  iteracion con costo:  4.193290101165257
800  iteracion con costo:  5.20523342803437
900  iteracion con costo:  4.8801625269292535
1000  iteracion con costo:  5.621068265760459
1100  iteracion con costo:  5.8864796755267355
1200  iteracion con costo:  5.981895342090294
1300  iteracion con costo:  5.27383476476275
1400  iteracion con costo:  4.345062780072135
1500  iteracion con costo:  5.448558284915103
1600  iteracion con costo:  4.520611357388399
1700  iteracion con costo:  5.379266464544956
1800  iteracion con costo:  4.78078436277266
1900  iteracion con costo:  5.09636362174765


In [20]:
ford = forward(X, W1, W2)
out = ford['a3']

In [27]:
matrix = np.zeros((10,10))
predic = np.argmax(out,axis=1)
out

array([[0.31861774, 0.18337279, 0.14433337, ..., 0.32320799, 0.31852503,
        0.00178437],
       [0.31861774, 0.18337279, 0.14433337, ..., 0.32320799, 0.31852503,
        0.00178437],
       [0.31861774, 0.18337279, 0.14433337, ..., 0.32320799, 0.31852503,
        0.00178437],
       ...,
       [0.31861773, 0.1833728 , 0.14433338, ..., 0.323208  , 0.31852506,
        0.00178438],
       [0.31861774, 0.18337279, 0.14433337, ..., 0.32320799, 0.31852503,
        0.00178437],
       [0.31861774, 0.18337279, 0.14433337, ..., 0.32320799, 0.31852503,
        0.00178437]])

In [23]:
mat_cross = np.zeros((10,10))
prediccion = np.zeros((5000,1))
for i in range(0,len(out)): #5000
    for j in range(0,len(out[i])): #10
        if out[i][j] > 0.5:
            prediccion[i][0] = j + 1
            
for i in range(0,len(prediccion)):
    _j = int(Y[i]-1) #valor real
    _i = int(prediccion[i][0]-1) #valor predicho
    mat_cross[_i][_j] += 1
    
print(mat_cross)

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [500. 500. 500. 500. 500. 500. 500. 500. 500. 500.]]
